## Purpose for Deployment

Deploying a social media website is essential for making it accessible to users beyond a developer’s local machine. Unlike running on localhost, deployment essential expands the user base web application meaning more people can access it. A deployed backend ensures the database can handle multiple users simultaneously, while cloud-based hosting enables scalability to support growing traffic. Additionally, deployment provides security features like HTTPS encryption and authentication, protecting user data from unauthorized access. 

## Deployment Steps

Prerequisites
- Review the backend-to-frontend functionality
- Document the deployment process with key steps and visuals
- Assign deployment admin roles
- Ensure configuration files (Dockerfile, docker-compose.yml, nginx config) are set 

<hr>

Local Testing Before Deployment
- Verify the backend is fully functional locally
- Test CRUD operations using API calls from postman
- Confirm the frontend correctly interacts with the backend

<hr>

Domain and Subdomain Setup
- Register a subdomain using AWS Route 53
- Configure the subdomain to route traffic correctly
```yml
yml
Server: https://clubhub_backend.nighthawkcodingsociety.com/
Domain: nighthawkcodingsociety.com
Subdomain: clubhub_backend
```
<hr>

Selecting Ports

- Choose an available backend port (we chose 8201 which doesn't conflict with classmate's ports)
- Update the port across all necessary files (example in main.py using new port number 8201)
```python
if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port="8201")
```
<hr>

Backend configuration
- Docker-related files
```Dockerfile
FROM docker.io/python:3.11
WORKDIR /
RUN apt-get update && apt-get upgrade -y && \
    apt-get install -y python3 python3-pip git
COPY . /
RUN pip install --no-cache-dir -r requirements.txt
RUN pip install gunicorn
ENV GUNICORN_CMD_ARGS="--workers=1 --bind=0.0.0.0:8201"
EXPOSE 8212
ENV FLASK_ENV=production
CMD [ "gunicorn", "main:app" ]
```
- docker-compose.yml
```yml
version: '3'
services:
        web:
                image: clubhub
                build: .
                env_file:
                        - .env # This file is optional; defaults will be used if it does not exist
                ports:
                        - "8201:8201"
                volumes:
                        - ./instance:/instance
                restart: unless-stopped
```
- nginx_file : preparing this file for reverse proxy (essentially sending info form internet, to application, and back to requestor)
```python
server {
    listen 80;
    listen [::]:80;
    server_name clubhub.stu.nighthawkcodingsociety.com ; # Change server name to the one on R53
    # Configure CORS Headers
    location / { 
        proxy_pass http://localhost:8201; # Change port to port on docker
        # Simple requests
        if ($request_method ~* "(GET|POST|PUT|DELETE)") { # Customize Request methods based on your needs
                add_header "Access-Control-Allow-Origin"  *;
        }
        # Preflighted requests 
        if ($request_method = OPTIONS ) {
                add_header "Access-Control-Allow-Origin"  "https://github.com/magic005/Club_Hub_Frontend" always;
                add_header "Access-Control-Allow-Methods" "GET, POST, PUT, DELETE, OPTIONS, HEAD"; # Make sure the request methods above match here
                add_header "Access-Control-Allow-Headers" "Authorization, Origin, X-Requested-With, Content-Type, Accept";
                return 200;
        }
    }
}
```
<hr>

Frontend settings: prepare the frontend to access our domain and ports to match our localhost, port 8201, and domain settings. this example is modifications made in config.js file on frontend
```python
export var pythonURI;
if (location.hostname === "localhost") {
        pythonURI = "http://localhost:8201";
} else if (location.hostname === "127.0.0.1") {
        pythonURI = "http://127.0.0.1:8201";
} else {
        pythonURI =  "clubhub.stu.nighthawkcodingsociety.com";
}
```
<hr>

Setting Up AWS EC2 Instance
- Log in to AWS and navigate to EC2
- Launch an appropriate EC2 instance (CSP)
- Connect to the instance via SS

Preparing Docker on Localhost

- Open the backend project in VSCode
- Ensure Docker files are correctly configured
- Run Docker locally and verify the service is accessible
- Push the latest changes to GitHub
<hr>

nginx Setup
1. ***Navigate to nginx***: cd /etc/nginx/sites-available<br>
2. ***Create an nginx config file***: sudo nano clubhub_backend<br>
3. ***Activate configuration***: cd /etc/nginx/sites-enabled, then sudo ln -s /etc/nginx/sites-available/clubhub_backend /etc/nginx/sites-enabled<br>
4. ***Validate***: sudo nginx -t<br>
5. ***Restart nginx***: sudo systemctl restart nginx<br>

Deploying to AWS EC2

1. Clone backend repo: git clone https://github.com/luojonah/clubhub_backend.git
2. Navigate to repo: cd clubhub_backend
3. Build site: docker-compose up -d --build
4. Test site: curl localhost:8201

Run command below and follow prompts:

sudo certbot --nginx


Updating and Redeploying Code
On Local Machine:

1. Run git pull before making changes
2. Open terminal in VSCode and run python main.py
3. Make changes that are needed
4. Commit the changes locally
5. Test docker-compose up or sudo docker-compose up in VSCode terminal
6. Sync change from UI/git push from terminal

Pulling Changes into AWS EC2 deployment:

1. Navigate to repo: cd ~/clubhub_backend
2. docker-compose down
3. git pull
4. Rebuild docker container: docker-compose up -d --build

Troubleshooting Deployment Issues

1. Check if the application is running correctly
2. Verify running Docker containers
3. Inspect logs for errors
4. Confirm proper port forwarding and domain configuration

## EWRI (errors we ran into)

1. make files, jupyter files and other miscellaneous files still used flocker_frontend url instead of Club_Hub_Frontend url. caused failures to deploying frontend on github pages but simple fix by replacing any function that used to this: "http://127.0.0.1:4887/flocker_frontend/home.html/" 

    with an updated url: "http://127.0.0.1:4887/Club_Hub_Frontend/home.html/"

2. CORS Errors when trying to fetch information with backend rendering our website useless. to fix this we first tested using a CORS browser extension and found out that CORS wasn't our error, but it had to do with the way we designed our token_required authenticators in the backend. 

    simple fix by removing our defined authentication function with an import:
    ```python
    from api.jwt_authorize import token_required
    ``` 
    and replacing @token_required decorators with @token_required() functions, and also removing authentication from the READ functionality.

3. login wasn't working on backend webpage. fixed by updating the structure of our nginx file to what Mort sent on slack, then re-running commands from our blog on cockpit to "re-deploy" the backend.


![Image](https://github.com/user-attachments/assets/f697feb5-3aec-47eb-9854-20a1c3c4ce58)

# Sending data through Internet

Packet: small amount of data send over a network. (Includes the source and destination info)

OSI: "Open" Systems Inteconnect 

Protocols
1. Physical
2. data link
3. network
4. transport
5. session
6. presentation
7. application

TCP - Transmission Control Protocol
- establishes common stnadard to sending messages

protocols
1. network access
2. Internet
3. Transport
4. application

Network Access Layer
- a lot to do with hardware
- Mac adress (unique)
- 0's and 1's 

Internet Layer Data Transmission
- obtain IP
- contains metadata
- taking path
- scalable (able to cahgne size)

Transport Layer
- sending an IP adress

TCP
- reliable
- slow

UDP
- fast
- not fully reliable

Application Layer

DNS - Domain Name Service
- maps numbers into names
- ex) .edu .org

World Wide Web
- not internet
- linked data pages

![Image](https://github.com/user-attachments/assets/8d2ced0e-3b13-464d-b31d-fb4097200b76)


Parallel Computing: schedules tasks to be executed at the same time

- faster
- a lot of data can be processed

Sequencial computing: Tasks are done one after another. 

- tasks are dependent
- have to execute one by one

Distributed Computing: sending of tasks from one computer to one or more others. 

- multiple devices
- solve that could not be solved on a single computer bc of the processing time or storage


